In [141]:
import json
import pandas as pd
import re
import string

In [168]:
data_path = "../dataset/Corona2.json"

with open(data_path,"r") as f:
    data = json.load(f)

In [169]:
data['examples'][1]['content']

'Diarrhea, also spelled diarrhoea, is the condition of having at least three loose, liquid, or watery bowel movements each day.[2] It often lasts for a few days and can result in dehydration due to fluid loss.[2] Signs of dehydration often begin with loss of the normal stretchiness of the skin and irritable behaviour.[2] This can progress to decreased urination, loss of skin color, a fast heart rate, and a decrease in responsiveness as it becomes more severe.[2] Loose but non-watery stools in babies who are exclusively breastfed, however, are normal.[2]'

In [170]:
len(data['examples'])

31

In [171]:
data['examples'][1]['annotations'][0]

{'id': '28601a42-c8a9-44e2-aeea-8939cb1db1a9',
 'tag_id': '03eb3e50-d4d8-4261-a60b-fa5aee5deb4a',
 'end': 382,
 'start': 364,
 'example_id': '487c93e3-0d45-4088-a378-cf3a01c8953d',
 'tag_name': 'MedicalCondition',
 'value': 'loss of skin color',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:23:08.961000Z',
   'annotator_id': 1,
   'tagged_token_id': '28601a42-c8a9-44e2-aeea-8939cb1db1a9',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [172]:
tags = []
for annots in data['examples']:
    annotations = [annotation['tag_name'] for annotation in annots['annotations']]
    tags.extend(annotations)

tags = list(set(tags))
tags

['MedicalCondition', 'Medicine', 'Pathogen']

In [173]:
bio_tags = []
for tag in tags:
    bio_tags.append(f"B-{tag}")
    bio_tags.append(f"I-{tag}")
bio_tags

['B-MedicalCondition',
 'I-MedicalCondition',
 'B-Medicine',
 'I-Medicine',
 'B-Pathogen',
 'I-Pathogen']

In [247]:
for annots in data['examples']:
    for annotation in annots['annotations']:
        if annotation['value'] == ' ':
            print(annotation)
            break
        print(annotation['value'],annotation['tag_name'])
    break

Diosmectite Medicine
aluminomagnesium silicate Medicine
diarrhea MedicalCondition
kaopectate Medicine
bismuth compounds Medicine
Pepto-Bismol Medicine
diarrhea  MedicalCondition
chemotherapy Medicine
constipation MedicalCondition
loperamide Medicine
diarrhea MedicalCondition
flatulence MedicalCondition
loperamide Medicine
{'id': 'a0a03c7b-cfad-41ee-9f5c-f8a802475994', 'tag_id': '03eb3e50-d4d8-4261-a60b-fa5aee5deb4a', 'end': 853, 'start': 852, 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe', 'tag_name': 'MedicalCondition', 'value': ' ', 'correct': None, 'human_annotations': [], 'model_annotations': []}


In [222]:
df_new = pd.DataFrame(columns=['id','content','labels'])
for annots in data['examples']:
    # print(f"{annots['id']}: {annots['content']}")
    labels = {annotation['value']:annotation['tag_name'] for annotation in annots['annotations']}
    # print("\n")
    # print(labels)
    new_labels = {}
    for label in labels:
        if len(label.split()) > 1:
            # i = 0
            for i,word in enumerate(label.split()):
                if i == 0:
                    new_labels[word] = f"B-{labels[label]}"
                else:
                    new_labels[word] = f"I-{labels[label]}"
        else:
            new_labels[label] = f"B-{labels[label]}"

    new_row = {'id': annots['id'], 'content': annots['content'], 'labels':new_labels}

    # Append the new row to the dataframe
    df_new.loc[len(df_new)] = new_row

In [223]:
df.head()

,id,content,labels
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,"{'Diosmectite': 'B-Medicine', 'aluminomagnesiu..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,"Diarrhea, also spelled diarrhoea, is the condi...","{'loss': 'B-MedicalCondition', 'of': 'I-Medica..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,"{'Antiretroviral': 'B-Medicine', 'therapy': 'I..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,"{'[8][5][93]': 'B-MedicalCondition', 'A': 'I-M..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,"The goals of treatment are to reduce pain, dec...","{'methotrexate': 'B-Medicine', 'hydroxychloroq..."


In [224]:
from nltk.tokenize import word_tokenize

text = '''While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]

Diosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.

Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]'''

word_tokenize(text)

['While',
 'bismuth',
 'compounds',
 '(',
 'Pepto-Bismol',
 ')',
 'decreased',
 'the',
 'number',
 'of',
 'bowel',
 'movements',
 'in',
 'those',
 'with',
 'travelers',
 "'",
 'diarrhea',
 ',',
 'they',
 'do',
 'not',
 'decrease',
 'the',
 'length',
 'of',
 'illness',
 '.',
 '[',
 '91',
 ']',
 'Anti-motility',
 'agents',
 'like',
 'loperamide',
 'are',
 'also',
 'effective',
 'at',
 'reducing',
 'the',
 'number',
 'of',
 'stools',
 'but',
 'not',
 'the',
 'duration',
 'of',
 'disease',
 '.',
 '[',
 '8',
 ']',
 'These',
 'agents',
 'should',
 'be',
 'used',
 'only',
 'if',
 'bloody',
 'diarrhea',
 'is',
 'not',
 'present',
 '.',
 '[',
 '92',
 ']',
 'Diosmectite',
 ',',
 'a',
 'natural',
 'aluminomagnesium',
 'silicate',
 'clay',
 ',',
 'is',
 'effective',
 'in',
 'alleviating',
 'symptoms',
 'of',
 'acute',
 'diarrhea',
 'in',
 'children',
 ',',
 '[',
 '93',
 ']',
 'and',
 'also',
 'has',
 'some',
 'effects',
 'in',
 'chronic',
 'functional',
 'diarrhea',
 ',',
 'radiation-induced',
 'd

In [225]:
def split_sentence(sentence):
    """
    Splits a sentence into a list of words while preserving words enclosed within brackets and detects 'None' values.

    Args:
        sentence (str): The sentence to split.

    Returns:
        list: A list of words.
    """
    words = re.split(r'((?<=\[)[^\]]+(?=\]))|[,.;:\s0-9]+', sentence)

    # Check for 'None' values and print warning messages
    for word in words:
        if word is None:
            print(f"Warning: 'None' value encountered in sentence: '{sentence}'")

    # Remove 'None' values from the list
    words = [word for word in words if word is not None]

    # Apply rstrip() to non-None values
    words = [word.rstrip() for word in words]

    return words

In [226]:
def remove_pattern(text):
    # Regular expression to match any number within brackets
    pattern_regex = re.compile('\[\d+\]')

    # Replace the pattern with an empty string
    text_without_pattern = pattern_regex.sub('', text)

    return text_without_pattern

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:3: SyntaxWarning: invalid escape sequence '\['
/var/folders/dm/s7p9m_4j3bgfqrvz0vd81ss40000gn/T/ipykernel_8321/2758500181.py:3: SyntaxWarning: invalid escape sequence '\['
  pattern_regex = re.compile('\[\d+\]')


In [227]:
def remove_punctuation(text):
    # Regular expression to match punctuation characters
    punctuation_regex = re.compile('[\\.,!?\'\"]')

    # Replace punctuation characters with an empty string
    text_without_punctuation = punctuation_regex.sub('', text)

    return text_without_punctuation

In [228]:
df_new['content'] = df_new['content'].apply(remove_pattern)
df_new['content'] = df_new['content'].apply(remove_punctuation)

In [229]:
df_new['sentence'] = df_new['content'].apply(split_sentence)


Diosmectite a natural aluminomagnesium silicate clay is effective in alleviating symptoms of acute diarrhea in children and also has some effects in chronic functional diarrhea radiation-induced diarrhea and chemotherapy-induced diarrhea Another absorbent agent used for the treatment of mild diarrhea is kaopectate

Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults It has better tolerability than loperamide as it causes less constipation and flatulence'

Diosmectite a natural aluminomagnesium silicate clay is effective in alleviating symptoms of acute diarrhea in children and also has some effects in chronic functional diarrhea radiation-induced diarrhea and chemotherapy-induced diarrhea Another absorbent agent used for the treatment of mild diarrhea is kaopectate

Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults It has better tolerability than loperamide as it causes less constipation and flatulen

In [230]:
df.head()

,id,content,labels,sentence
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,"{'Diosmectite': 'B-Medicine', 'aluminomagnesiu...","[While, bismuth, compounds, (Pepto-Bismol), de..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,Diarrhea also spelled diarrhoea is the conditi...,"{'loss': 'B-MedicalCondition', 'of': 'I-Medica...","[Diarrhea, also, spelled, diarrhoea, is, the, ..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,"{'Antiretroviral': 'B-Medicine', 'therapy': 'I...","[Antiretroviral, therapy, (ART), is, recommend..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,"{'[8][5][93]': 'B-MedicalCondition', 'A': 'I-M...","[The, following, drugs, are, considered, as, D..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,The goals of treatment are to reduce pain decr...,"{'methotrexate': 'B-Medicine', 'hydroxychloroq...","[The, goals, of, treatment, are, to, reduce, p..."


In [219]:
testing = {'Diosmectite': 'Medicine',
 'aluminomagnesium silicate': 'Medicine',
 'diarrhea': 'MedicalCondition',
 'kaopectate': 'Medicine',
 'bismuth compounds': 'Medicine',
 'Pepto-Bismol': 'Medicine',
 'diarrhea ': 'MedicalCondition',
 'chemotherapy': 'Medicine',
 'constipation': 'MedicalCondition',
 'loperamide': 'Medicine',
 'flatulence': 'MedicalCondition',
 ' ': 'MedicalCondition',
 'Racecadotril': 'Medicine'}

new_labels = {}
for label in testing:
    if len(label.split()) > 1:
        # i = 0
        for i,word in enumerate(label.split()):
            if i == 0:
                new_labels[word] = f"B-{testing[label]}"
            else:
                new_labels[word] = f"I-{testing[label]}"
    else:
        new_labels[label] = f"B-{testing[label]}"

new_labels
         


{'Diosmectite': 'B-Medicine',
 'aluminomagnesium': 'B-Medicine',
 'silicate': 'I-Medicine',
 'diarrhea': 'B-MedicalCondition',
 'kaopectate': 'B-Medicine',
 'bismuth': 'B-Medicine',
 'compounds': 'I-Medicine',
 'Pepto-Bismol': 'B-Medicine',
 'diarrhea ': 'B-MedicalCondition',
 'chemotherapy': 'B-Medicine',
 'constipation': 'B-MedicalCondition',
 'loperamide': 'B-Medicine',
 'flatulence': 'B-MedicalCondition',
 ' ': 'B-MedicalCondition',
 'Racecadotril': 'B-Medicine'}

In [218]:
testing

{'Diosmectite': 'Medicine',
 'aluminomagnesium silicate': 'Medicine',
 'diarrhea': 'MedicalCondition',
 'kaopectate': 'Medicine',
 'bismuth compounds': 'Medicine',
 'Pepto-Bismol': 'Medicine',
 'diarrhea ': 'MedicalCondition',
 'chemotherapy': 'Medicine',
 'constipation': 'MedicalCondition',
 'loperamide': 'Medicine',
 'flatulence': 'MedicalCondition',
 ' ': 'MedicalCondition',
 'Racecadotril': 'Medicine'}

In [184]:
tags

['MedicalCondition', 'Medicine', 'Pathogen']

In [185]:
len('praneeth'.split())

1

In [231]:
def create_bio_tags(row):
    BIO = []
    sentence = row['sentence']
    labels = row['labels']
    for word in sentence:
        if word in labels:
            BIO.append(labels[word])
        else:
            BIO.append("O")

    return BIO


In [232]:
df_new['bio_tags'] = df_new.apply(create_bio_tags,axis=1)

In [233]:
df.head()

,id,content,labels,sentence,bio_tags
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,"{'Diosmectite': 'B-Medicine', 'aluminomagnesiu...","[While, bismuth, compounds, (Pepto-Bismol), de...","[O, B-Medicine, I-Medicine, O, O, O, O, O, O, ..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,Diarrhea also spelled diarrhoea is the conditi...,"{'loss': 'B-MedicalCondition', 'of': 'I-Medica...","[Diarrhea, also, spelled, diarrhoea, is, the, ...","[B-MedicalCondition, O, O, B-MedicalCondition,..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,"{'Antiretroviral': 'B-Medicine', 'therapy': 'I...","[Antiretroviral, therapy, (ART), is, recommend...","[B-Medicine, I-Medicine, O, O, O, O, O, O, O, ..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,"{'[8][5][93]': 'B-MedicalCondition', 'A': 'I-M...","[The, following, drugs, are, considered, as, D...","[O, O, O, O, O, O, B-Medicine, B-Medicine, B-M..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,The goals of treatment are to reduce pain decr...,"{'methotrexate': 'B-Medicine', 'hydroxychloroq...","[The, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [234]:
' '.join(df[['sentence','bio_tags','labels']].iloc[0]['sentence'])

'While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers diarrhea they do not decrease the length of illness Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease These agents should be used only if bloody diarrhea is not present Diosmectite a natural aluminomagnesium silicate clay is effective in alleviating symptoms of acute diarrhea in children and also has some effects in chronic functional diarrhea radiation-induced diarrhea and chemotherapy-induced diarrhea Another absorbent agent used for the treatment of mild diarrhea is kaopectate Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults It has better tolerability than loperamide as it causes less constipation and flatulence'

In [235]:
"aluminomagnesium" in df[['sentence','bio_tags','labels']].iloc[0]['sentence']

True

In [238]:
df[['sentence','bio_tags']].iloc[0]['sentence'].index("silicate")

58

In [240]:
df[['sentence','bio_tags']].iloc[0]['bio_tags'][59]

'O'

In [242]:
df[['sentence','bio_tags','labels']].iloc[1]['labels']

{'loss': 'B-MedicalCondition',
 'of': 'I-MedicalCondition',
 'skin': 'I-MedicalCondition',
 'color': 'I-MedicalCondition',
 'Diarrhea': 'B-MedicalCondition',
 'watery': 'B-MedicalCondition',
 'bowel': 'I-MedicalCondition',
 'movements': 'I-MedicalCondition',
 'dehydration': 'B-MedicalCondition',
 'diarrhoea': 'B-MedicalCondition',
 'decrease': 'B-MedicalCondition',
 'in': 'I-MedicalCondition',
 'responsiveness': 'I-MedicalCondition',
 'fast': 'B-MedicalCondition',
 'heart': 'I-MedicalCondition',
 'rate': 'I-MedicalCondition'}

In [243]:
df.head()

,id,content,labels,sentence,bio_tags
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,"{'Diosmectite': 'B-Medicine', 'aluminomagnesiu...","[While, bismuth, compounds, (Pepto-Bismol), de...","[O, B-Medicine, I-Medicine, O, O, O, O, O, O, ..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,Diarrhea also spelled diarrhoea is the conditi...,"{'loss': 'B-MedicalCondition', 'of': 'I-Medica...","[Diarrhea, also, spelled, diarrhoea, is, the, ...","[B-MedicalCondition, O, O, B-MedicalCondition,..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,"{'Antiretroviral': 'B-Medicine', 'therapy': 'I...","[Antiretroviral, therapy, (ART), is, recommend...","[B-Medicine, I-Medicine, O, O, O, O, O, O, O, ..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,"{'[8][5][93]': 'B-MedicalCondition', 'A': 'I-M...","[The, following, drugs, are, considered, as, D...","[O, O, O, O, O, O, B-Medicine, B-Medicine, B-M..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,The goals of treatment are to reduce pain decr...,"{'methotrexate': 'B-Medicine', 'hydroxychloroq...","[The, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [255]:
df[df['id']=="18c2f619-f102-452f-ab81-d26f7e283ffe"]['labels'].values[0]

{'Diosmectite': 'B-Medicine',
 'aluminomagnesium': 'B-Medicine',
 'silicate': 'I-Medicine',
 'diarrhea': 'B-MedicalCondition',
 'kaopectate': 'B-Medicine',
 'bismuth': 'B-Medicine',
 'compounds': 'I-Medicine',
 'Pepto-Bismol': 'B-Medicine',
 'diarrhea ': 'B-MedicalCondition',
 'chemotherapy': 'B-Medicine',
 'constipation': 'B-MedicalCondition',
 'loperamide': 'B-Medicine',
 'flatulence': 'B-MedicalCondition',
 ' ': 'B-MedicalCondition',
 'Racecadotril': 'B-Medicine'}

In [257]:
df.head()

,id,content,labels,sentence,bio_tags
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,"{'Diosmectite': 'B-Medicine', 'aluminomagnesiu...","[While, bismuth, compounds, (Pepto-Bismol), de...","[O, B-Medicine, I-Medicine, O, O, O, O, O, O, ..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,Diarrhea also spelled diarrhoea is the conditi...,"{'loss': 'B-MedicalCondition', 'of': 'I-Medica...","[Diarrhea, also, spelled, diarrhoea, is, the, ...","[B-MedicalCondition, O, O, B-MedicalCondition,..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,"{'Antiretroviral': 'B-Medicine', 'therapy': 'I...","[Antiretroviral, therapy, (ART), is, recommend...","[B-Medicine, I-Medicine, O, O, O, O, O, O, O, ..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,"{'[8][5][93]': 'B-MedicalCondition', 'A': 'I-M...","[The, following, drugs, are, considered, as, D...","[O, O, O, O, O, O, B-Medicine, B-Medicine, B-M..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,The goals of treatment are to reduce pain decr...,"{'methotrexate': 'B-Medicine', 'hydroxychloroq...","[The, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [259]:
df[['id','sentence','bio_tags']]

,id,sentence,bio_tags
0,18c2f619-f102-452f-ab81-d26f7e283ffe,"[While, bismuth, compounds, (Pepto-Bismol), de...","[O, B-Medicine, I-Medicine, O, O, O, O, O, O, ..."
1,487c93e3-0d45-4088-a378-cf3a01c8953d,"[Diarrhea, also, spelled, diarrhoea, is, the, ...","[B-MedicalCondition, O, O, B-MedicalCondition,..."
2,d5056874-895a-4a7f-9e0f-828d414d65d9,"[Antiretroviral, therapy, (ART), is, recommend...","[B-Medicine, I-Medicine, O, O, O, O, O, O, O, ..."
3,20c792c7-0c4b-42d0-8127-0e04113db384,"[The, following, drugs, are, considered, as, D...","[O, O, O, O, O, O, B-Medicine, B-Medicine, B-M..."
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,"[The, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5,b3481879-a9cd-48db-8526-8ee11c9a72bc,"[Hantaviruses, usually, found, in, rodents, an...","[B-Pathogen, O, O, O, O, O, O, O, O, O, O, O, ..."
6,8ecb663a-c4d5-4351-9b9c-40f724ff936d,"[Bats, are, the, most, common, source, of, rab...","[O, O, O, O, O, O, O, B-Pathogen, O, O, O, O, ..."
7,f2f13903-be8b-4981-8702-4ee992f90534,"[In, following, the, outbreak, of, severe, acu...","[O, O, O, O, O, B-MedicalCondition, I-MedicalC..."
8,5d1621d3-b877-4747-aceb-ecf807a6c9ec,"[Bacterial, vaginosis, is, caused, by, bacteri...","[B-Pathogen, I-Pathogen, O, O, O, B-Pathogen, ..."
9,9a0b1ede-2282-4dae-b086-638e45881bf5,"[Other, groups, of, intracellular, bacterial, ...","[O, O, O, O, O, O, O, B-Pathogen, B-Pathogen, ..."


In [262]:
{value:key for key,value in enumerate(bio_tags)}

{'B-MedicalCondition': 0,
 'I-MedicalCondition': 1,
 'B-Medicine': 2,
 'I-Medicine': 3,
 'B-Pathogen': 4,
 'I-Pathogen': 5}

In [264]:
df.drop(['id','content','labels'],axis=1)

,sentence,bio_tags
0,"[While, bismuth, compounds, (Pepto-Bismol), de...","[O, B-Medicine, I-Medicine, O, O, O, O, O, O, ..."
1,"[Diarrhea, also, spelled, diarrhoea, is, the, ...","[B-MedicalCondition, O, O, B-MedicalCondition,..."
2,"[Antiretroviral, therapy, (ART), is, recommend...","[B-Medicine, I-Medicine, O, O, O, O, O, O, O, ..."
3,"[The, following, drugs, are, considered, as, D...","[O, O, O, O, O, O, B-Medicine, B-Medicine, B-M..."
4,"[The, goals, of, treatment, are, to, reduce, p...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5,"[Hantaviruses, usually, found, in, rodents, an...","[B-Pathogen, O, O, O, O, O, O, O, O, O, O, O, ..."
6,"[Bats, are, the, most, common, source, of, rab...","[O, O, O, O, O, O, O, B-Pathogen, O, O, O, O, ..."
7,"[In, following, the, outbreak, of, severe, acu...","[O, O, O, O, O, B-MedicalCondition, I-MedicalC..."
8,"[Bacterial, vaginosis, is, caused, by, bacteri...","[B-Pathogen, I-Pathogen, O, O, O, B-Pathogen, ..."
9,"[Other, groups, of, intracellular, bacterial, ...","[O, O, O, O, O, O, O, B-Pathogen, B-Pathogen, ..."
